In [1]:
import numpy as np
import scipy.stats as stat
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../../data/summary/site_month.csv')
data

,dt,global,tropical,south,north
0,1900-01-01,11.555170,22.977459,19.990000,2.953946
1,1900-02-01,13.583870,24.118351,20.416714,5.769482
2,1900-03-01,16.425790,25.646919,19.448714,9.955393
3,1900-04-01,19.520930,26.158541,17.071000,15.441607
4,1900-05-01,22.025110,26.093838,15.254000,20.183232
...,...,...,...,...,...
1360,2013-05-01,23.405960,26.979703,15.773429,21.998804
1361,2013-06-01,24.341760,25.659054,14.429286,24.710464
1362,2013-07-01,24.951320,24.746486,13.986286,26.457286
1363,2013-08-01,24.770230,24.867730,14.411571,26.000643


In [11]:
ttest = pd.DataFrame()
target = data.iloc[-120:]['global']
s = []
p = []
idx = []

for i in range(48, 1365, 120):
    tem1, tem2 = stat.ttest_ind(
        data.iloc[i: 120 + i]['global'], target, alternative='two-sided'
    )
    idx.append(data.dt[i])
    s.append(tem1)
    p.append(tem2)

ttest.index = idx
ttest['stat'] = s
ttest['p'] = p

ttest = ttest[:-1]
ttest

,stat,p
1904-01-01,-2.207756,0.028216
1914-01-01,-1.841504,0.066792
1924-01-01,-1.605102,0.109797
1934-01-01,-1.376537,0.169949
1944-01-01,-1.265140,0.207059
1954-01-01,-1.291485,0.197789
1964-01-01,-1.362050,0.174470
1974-01-01,-1.183818,0.237666
1984-01-01,-0.875372,0.382254
1994-01-01,-0.204696,0.837985


In [4]:
stat.ttest_ind(
        data.iloc[3: 13]['global'], target, alternative='two-sided'
    )[0]

0.06868147471668457

In [12]:
ttest.to_csv('../../data/summary/ttest.csv')

In [13]:
from neuralprophet import NeuralProphet
import joblib

npt: NeuralProphet = joblib.load('../../models/model1/np.model')
npt.fitted

True

In [20]:
X = data[['dt', 'global']]
X.columns = ['ds', 'y']
y = npt.predict(X)
y

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 91.575% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to 91.581% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS


,ds,y,yhat1,residual1,ar1,trend,season_yearly
0,1900-01-01,11.555170,None,NaN,None,None,None
1,1900-02-01,13.583870,None,NaN,None,None,None
2,1900-03-01,16.425790,None,NaN,None,None,None
3,1900-04-01,19.520930,None,NaN,None,None,None
4,1900-05-01,22.025110,None,NaN,None,None,None
...,...,...,...,...,...,...,...
1360,2013-05-01,23.405960,23.146599,-0.259361,0.0,19.794018,3.352582
1361,2013-06-01,24.341760,24.518566,0.176806,0.0,19.795662,4.722902
1362,2013-07-01,24.951320,25.01931,0.06799,0.0,19.797256,5.222052
1363,2013-08-01,24.770230,24.755905,-0.014325,0.0,19.798903,4.957003


In [21]:
y[['ds', 'trend', 'season_yearly']].to_csv('../../data/summary/period.csv')